# SAT Tutor - Fine-tuned with Gemma 3N

This project fine-tunes **Gemma 3N** to create a subject-specific SAT tutoring assistant, capable of solving and explaining math and reasoning problems.

Special thanks to **Unsloth AI** for their excellent [Gemma 3N fine-tuning notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3N_(4B)-Conversational.ipynb), which served as the foundation for this workflow.


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

### Unsloth

`FastModel` supports loading any Gemma model.

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False,
)

# Finetuning Gemma 3N!

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # Should leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [GSM8k](https://huggingface.co/datasets/openai/gsm8k), [RACE](https://huggingface.co/datasets/ehovy/race), [openbookqa](https://huggingface.co/datasets/allenai/openbookqa), and [truthful_qa](https://huggingface.co/datasets/truthfulqa/truthfulqa) datasets in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template.

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

Loading and merging all datasets.

In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load datasets
dataset1 = load_dataset("gsm8k", "main", split="train")
dataset2 = load_dataset("ehovy/race", "all", split="train")
dataset3 = load_dataset("allenai/openbookqa", "main", split="train")
dataset4 = load_dataset("truthfulqa/truthful_qa", "generation", split="validation")

# Combine all datasets
dataset = concatenate_datasets([dataset1, dataset2, dataset3, dataset4])

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [ ]:
from datasets import Dataset
from typing import Union

def preprocess_dataset(dataset: Union[Dataset, list], seed: int = 42):
    """
    Filters out bad rows (empty questions/answers), removes duplicates, and shuffles the dataset.
    Assumes 'question' and 'answer' fields exist.
    """

    # Step 1: Check and filter out empty question or answer
    print("Checking and filtering empty or invalid questions/answers...")
    dataset = dataset.filter(
        lambda x: isinstance(x["question"], str) and x["question"].strip() != "" and
                  isinstance(x["answer"], str) and x["answer"].strip() != ""
    )

    # Step 2: Deduplicate using string key of question + answer
    seen = set()
    def dedup(example):
        key = (example["question"].strip(), example["answer"].strip())
        if key in seen:
            return False
        seen.add(key)
        return True

    print("Removing duplicates...")
    dataset = dataset.filter(dedup)

    # Step 3: Shuffle
    print("Shuffling dataset...")
    dataset = dataset.shuffle(seed=seed)

    print(f"Final dataset size: {len(dataset)}")
    return dataset

dataset = preprocess_dataset(dataset)

Let's see how row 100 looks like!

In [ ]:
dataset[100]

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [ ]:
def formatting_prompts_func(examples):
    texts = []
    for q, a in zip(examples["question"], examples["answer"]):
        # Ensure both question and answer are strings, defaulting to empty string if None
        q = str(q) if q is not None else ""
        a = str(a) if a is not None else ""

        chat = [
            {"role": "user", "content": q},
            {"role": "assistant", "content": a}
        ]
        try:
            text = tokenizer.apply_chat_template(
                chat,
                tokenize=False,
                add_generation_prompt=False
            ).removeprefix("<bos>")
        except Exception as e:
            print(f"Skipping due to error: {e}")
            continue
        texts.append(text)

    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [ ]:
dataset[100]["text"]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! We do 100 steps.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-5,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Let's train the model!

To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

### Large Losses During Fine-tuning

High initial training losses (e.g., 6–18) are expected when fine-tuning models like **Gemma 3N**, especially due to its multimodal nature. The loss typically drops and stabilizes quickly, which aligns with what I observed.

For more details, see [Unsloth's explanation](https://unsloth.ai/blog/gemma-3n#large-losses-during-finetuning).

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template

# Re-attach the correct chat template (same as used for training)
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

# Format your inference prompt
messages = [
    {
        "role": "user",
        "content": [{
            "type": "text",
            "text": "A square has an area of 49 square centimeters. What is the perimeter of the square?",
        }]
    }
]

# Apply template for generation
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Required for generation!
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
).to("cuda")

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
)

# Decode and print the model's response
tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# Define the prompt in Unsloth-compatible format
messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "A triangle has angles measuring 35° and 75°. What is the measure of the third angle?",
    }]
}]

# Tokenize the prompt
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,  # Appends <start_of_turn>model\n
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
# Generate and stream
_ = model.generate(
    **inputs,
    max_new_tokens = 256,
    temperature = 1.0,
    top_p = 0.95,
    top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, use `save_pretrained` for a local save.

#### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, Unsloth supports it natively now for all models. We can convert easily to `Q8_0, F16 or BF16` precision.

In [ ]:
model.save_pretrained_merged("sat-tutor-gemma3n-merged", tokenizer)

model.save_pretrained_gguf(
    "sat-tutor-gemma3n-merged",  # must be the merged folder
    quantization_type="Q8_0",    # or "F16"
)

In [ ]:
# Save the model as a zip file and download
!zip sat-tutor-gemma3n.Q8_0.zip /content/sat-tutor-gemma3n-merged.Q8_0.gguf
from google.colab import files
files.download("sat-tutor-gemma3n.Q8_0.zip")

Now, use the `sat-tutor-gemma3n-merged.Q8_0.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI.